# BIGRoC: Boosting Image Generation via a Robust Classifier

--------------------------------------------------

## Boosting Guided Diffusion on ImageNet $256\times256$

This colab notebook contains the needed code to experiment with our proposed algorithm for boosting Guided Diffusion on ImageNet 256x256(in Section 5.2).

**How to use**

1.   Upload the notebook to colab
2.   Make sure that colab uses a GPU (Edit $\rightarrow$ Notebook settings $\rightarrow$ Hardware accelerator)

**Setup:**

This notebook mounts google drive and assumes that [this directory](https://drive.google.com/drive/folders/1yN6WjMmc-pi3zHylF-I1jri7I2nZpsGJ?usp=sharing) is located in your root folder in Google Drive. Therefore, please open the Google Drive's link and choose "add a shortcut to drive" and pick "My Drive" is the chosen location. By doing so, you are ready to go :)

If you rather not to mount your google drive, download the files and edit the relevant paths accordingly.

**Credits**

This notebook uses the following packages:

* [robustness](https://github.com/MadryLab/robustness) - For adversarially robust classifier
* [guided-diffuision](https://github.com/openai/guided-diffusion/tree/main/evaluations) - ImageNet quantitative evaluation and 50K set of generated images + labels

# Setup & Installation

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install torch==1.8.2+cu102 torchvision==0.9.2+cu102 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html

Looking in links: https://download.pytorch.org/whl/lts/1.8/torch_lts.html
     |████████████████████████████████| 804.1 MB 9.0 kB/s 
     |████████████████████████████████| 17.3 MB 2.2 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.8.2+cu102 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.8.2+cu102 which is incompatible.


In [ ]:
!wget http://andrewilyas.com/ImageNet.pt

--2022-02-03 06:51:37--  http://andrewilyas.com/ImageNet.pt
Resolving andrewilyas.com (andrewilyas.com)... 52.218.182.146
Connecting to andrewilyas.com (andrewilyas.com)|52.218.182.146|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 204818881 (195M) [binary/octet-stream]
Saving to: ‘ImageNet.pt’

ImageNet.pt         100%[===================>] 195.33M  35.5MB/s    in 6.2s    

2022-02-03 06:51:44 (31.7 MB/s) - ‘ImageNet.pt’ saved [204818881/204818881]



In [ ]:
!pip install robustness

     |████████████████████████████████| 95 kB 2.6 MB/s 
     |████████████████████████████████| 180 kB 30.1 MB/s 
     |████████████████████████████████| 124 kB 65.7 MB/s 
     |████████████████████████████████| 55 kB 3.4 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=f761d419fd1595ccff90e0e9023d8fbf103934e9834bd86729423fe9ec12c193
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built GPUtil


In [ ]:
from robustness import model_utils, datasets
import torch


def create_dl_model(DATA='CIFAR', BATCH_SIZE=128, NUM_WORKERS=8):
    '''
    :param DATA: Choices: ['CIFAR', 'ImageNet', 'RestrictedImageNet']
    :param bs: batch size
    :param num_workers:
    :return: a dataloader object
    '''

    # Load dataset
    dataset_function = getattr(datasets, DATA)
    dataset = dataset_function('data')
    # Load model
    model_kwargs = {
        'arch': 'resnet50',
        'dataset': dataset,
        'resume_path': f'./{DATA}.pt'
    }
    model, _ = model_utils.make_and_restore_model(**model_kwargs)
    classifier = model.model
    classifier.eval()
    return classifier

adv_model = create_dl_model(DATA='ImageNet')

=> loading checkpoint './ImageNet.pt'
=> loaded checkpoint './ImageNet.pt' (epoch 105)


In [ ]:
import torch
from torch import nn


class AttackerStep:
    '''
    Generic class for attacker steps, under perturbation constraints
    specified by an "origin input" and a perturbation magnitude.
    Must implement project, step, and random_perturb
    '''

    def __init__(self, orig_input, eps, step_size, use_grad=True):
        '''
        Initialize the attacker step with a given perturbation magnitude.
        Args:
            eps (float): the perturbation magnitude
            orig_input (ch.tensor): the original input
        '''
        self.orig_input = orig_input
        self.eps = eps
        self.step_size = step_size
        self.use_grad = use_grad

    def project(self, x):
        '''
        Given an input x, project it back into the feasible set
        Args:
            ch.tensor x : the input to project back into the feasible set.
        Returns:
            A `ch.tensor` that is the input projected back into
            the feasible set, that is,
        .. math:: \min_{x' \in S} \|x' - x\|_2
        '''
        raise NotImplementedError

    def step(self, x, g):
        '''
        Given a gradient, make the appropriate step according to the
        perturbation constraint (e.g. dual norm maximization for :math:`\ell_p`
        norms).
        Parameters:
            g (ch.tensor): the raw gradient
        Returns:
            The new input, a ch.tensor for the next step.
        '''
        raise NotImplementedError

    def random_perturb(self, x):
        '''
        Given a starting input, take a random step within the feasible set
        '''
        raise NotImplementedError

    def to_image(self, x):
        '''
        Given an input (which may be in an alternative parameterization),
        convert it to a valid image (this is implemented as the identity
        function by default as most of the time we use the pixel
        parameterization, but for alternative parameterizations this functino
        must be overriden).
        '''
        return x


from torch import nn

# simple Module to normalize an image
class Normalize(nn.Module):
    def __init__(self, mean, std):
        super(Normalize, self).__init__()
        self.mean = torch.Tensor(mean)
        self.std = torch.Tensor(std)
    def forward(self, x):
        return (x - self.mean.type_as(x)[None, :, None, None]) / self.std.type_as(x)[None, :, None, None]

norm = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])



# L2 threat model
class L2Step(AttackerStep):
    """
    Attack step for :math:`\ell_\infty` threat model. Given :math:`x_0`
    and :math:`\epsilon`, the constraint set is given by:
    .. math:: S = \{x | \|x - x_0\|_2 \leq \epsilon\}
    """

    def project(self, x):
        """
        """
        if self.orig_input is None: self.orig_input = x.detach()
        self.orig_input = self.orig_input.cuda()
        diff = x - self.orig_input
        diff = diff.renorm(p=2, dim=0, maxnorm=self.eps)
        return torch.clamp(self.orig_input + diff, 0, 1)

    def step(self, x, g):
        """
        """
        l = len(x.shape) - 1
        g_norm = torch.norm(g.reshape(g.shape[0], -1), dim=1).view(-1, *([1] * l))
        scaled_g = g / (g_norm + 1e-10)
        return x + scaled_g * self.step_size

def targeted_pgd_l2(model, X, y, num_iter, eps, step_size):
    # input images are in range [0,1]
    steper = L2Step(eps=eps, orig_input=None, step_size=step_size)
    for t in range(num_iter):
        X = X.clone().detach().requires_grad_(True).cuda()
        loss = nn.CrossEntropyLoss(reduction='none')(model(norm(X)), y)
        loss = torch.mean(loss)
        grad, = torch.autograd.grad(-1 * loss, [X])
        X = steper.step(X, grad)
        X = steper.project(X)
    return X.detach()

In [ ]:
!wget https://openaipublic.blob.core.windows.net/diffusion/jul-2021/ref_batches/imagenet/256/admnet_guided_upsampled_imagenet256.npz

--2022-02-03 06:51:57--  https://openaipublic.blob.core.windows.net/diffusion/jul-2021/ref_batches/imagenet/256/admnet_guided_upsampled_imagenet256.npz
Resolving openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)... 20.60.241.33
Connecting to openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)|20.60.241.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9830800614 (9.2G) [application/octet-stream]
Saving to: ‘admnet_guided_upsampled_imagenet256.npz’

admnet_guided_upsam 100%[===================>]   9.16G  30.7MB/s    in 4m 55s  

2022-02-03 06:56:52 (31.8 MB/s) - ‘admnet_guided_upsampled_imagenet256.npz’ saved [9830800614/9830800614]



In [ ]:
import numpy as np

x = np.load("admnet_guided_upsampled_imagenet256.npz", mmap_mode='r') 
arr_orig = x['arr_0']
gt_labels = x['arr_1']

In [ ]:
arr_orig.shape, arr_orig.min(), arr_orig.max()

((50000, 256, 256, 3), 0, 255)

Download Reference Batch

In [ ]:
!wget https://openaipublic.blob.core.windows.net/diffusion/jul-2021/ref_batches/imagenet/256/VIRTUAL_imagenet256_labeled.npz

--2022-02-03 06:57:34--  https://openaipublic.blob.core.windows.net/diffusion/jul-2021/ref_batches/imagenet/256/VIRTUAL_imagenet256_labeled.npz
Resolving openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)... 20.60.241.33
Connecting to openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)|20.60.241.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2037122530 (1.9G) [application/octet-stream]
Saving to: ‘VIRTUAL_imagenet256_labeled.npz’

VIRTUAL_imagenet256 100%[===================>]   1.90G  23.2MB/s    in 63s     

2022-02-03 06:58:37 (30.7 MB/s) - ‘VIRTUAL_imagenet256_labeled.npz’ saved [2037122530/2037122530]



# Guided Diffusion baseline quantitative results

ADM-G+U Eval

In [ ]:
!python ./gdrive/MyDrive/models_and_scripts/evaluator.py ./VIRTUAL_imagenet256_labeled.npz ./admnet_guided_upsampled_imagenet256.npz

downloading InceptionV3 model...
11679it [00:16, 722.62it/s]
2022-02-01 14:20:19.321184: W tensorflow/core/framework/op_def_util.cc:371] Op BatchNormWithGlobalNormalization is deprecated. It will cease to work in GraphDef version 9. Use tf.nn.batch_normalization().
warming up TensorFlow...
100% 1/1 [00:08<00:00,  8.37s/it]
computing reference batch activations...
100% 157/157 [00:34<00:00,  4.60it/s]
computing/reading reference batch statistics...
computing sample batch activations...
100% 782/782 [02:44<00:00,  4.76it/s]
computing/reading sample batch statistics...
Computing evaluations...
Inception Score: 215.837158203125
FID: 3.9425602260648134
sFID: 6.140429580212867
tcmalloc: large alloc 2000003072 bytes == 0x5650dd5b8000 @  0x7fd97f7c0001 0x7fd97c28854f 0x7fd97c2d8b58 0x7fd97c2dcb17 0x7fd97c37b203 0x56500b44a544 0x56500b44a240 0x56500b4be627 0x56500b4b8ced 0x56500b44bbda 0x56500b4b9c0d 0x56500b44bafa 0x56500b4b9c0d 0x56500b44bafa 0x56500b4b9915 0x56500b4b89ee 0x56500b4b86f3 0x565

ADM-G eval

In [ ]:
!python ./gdrive/MyDrive/models_and_scripts/evaluator.py ./VIRTUAL_imagenet256_labeled.npz ./admnet_guided_imagenet256.npz

downloading InceptionV3 model...
11679it [00:05, 2174.31it/s]
2022-01-13 16:06:56.476211: W tensorflow/core/framework/op_def_util.cc:371] Op BatchNormWithGlobalNormalization is deprecated. It will cease to work in GraphDef version 9. Use tf.nn.batch_normalization().
warming up TensorFlow...
100% 1/1 [00:08<00:00,  8.25s/it]
computing reference batch activations...
100% 157/157 [00:32<00:00,  4.77it/s]
computing/reading reference batch statistics...
computing sample batch activations...
100% 782/782 [02:45<00:00,  4.73it/s]
computing/reading sample batch statistics...
Computing evaluations...
Inception Score: 186.69859313964844
FID: 4.586654813293535
sFID: 5.24677393057334
tcmalloc: large alloc 2000003072 bytes == 0x55b92cd0a000 @  0x7fb7cb766001 0x7fb7c822e54f 0x7fb7c827eb58 0x7fb7c8282b17 0x7fb7c8321203 0x55b85953b544 0x55b85953b240 0x55b8595af627 0x55b8595a9ced 0x55b85953cbda 0x55b8595aac0d 0x55b85953cafa 0x55b8595aac0d 0x55b85953cafa 0x55b8595aa915 0x55b8595a99ee 0x55b8595a96f3 0x55

# BIGRoC Application

In [ ]:
# BIGRoC args
epsilon, steps = 1, 7
step_size = (epsilon * 1.5) / steps

In [ ]:
from tqdm.notebook import tqdm

torch.manual_seed(1234)
for k in range(50):
  boosted_gen_imgs = []
  for i in tqdm(range(100)):
    x_batch = arr_orig[(1000*k) + i * 10: (1000*k) + (i + 1) * 10]
    x_batch_0_1 = torch.tensor(x_batch / 255.).cuda().permute(0,3,1,2).float()
    with torch.no_grad():
      # labels = torch.argmax(adv_model(norm(x_batch_0_1)), dim=1)
      labels = torch.tensor(gt_labels[(1000*k) + i * 10: (1000*k) + (i + 1) * 10]).cuda().long()
    b_imgs = targeted_pgd_l2(model=adv_model, X=x_batch_0_1.data, y=labels.long(), num_iter=steps, eps=epsilon,
                                      step_size=step_size).detach().cpu()
    boosted_gen_imgs.append(b_imgs)

  boosted_gen_imgs = torch.cat(boosted_gen_imgs)
  boosted_gen_imgs = (boosted_gen_imgs * 255.).int()
  boosted_gen_imgs = boosted_gen_imgs.detach().cpu().permute(0,2,3,1).numpy().astype(np.uint8)
  np.savez(f"./boosted_diff_256_{k}_eps_{epsilon}_steps_{steps}_labeled", boosted_gen_imgs)
  # np.savez(f"./boosted_diff_256_{k}_eps_{epsilon}_steps_{steps}", boosted_gen_imgs)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

load boosted images

In [ ]:
import numpy as np
from tqdm.notebook import tqdm

arr = np.zeros(shape=(50000, 256, 256, 3), dtype=np.uint8)

for f in tqdm(range(50)):
  x = np.load(f"./boosted_diff_256_{f}_eps_{epsilon}_steps_{steps}_labeled.npz")['arr_0']
  # x = np.load(f"./boosted_diff_256_{f}_eps_{epsilon}_steps_{steps}.npz")['arr_0']
  arr[1000 * f: 1000 * (f+1)] = x
  x = None

np.savez(f"boosted_diff_256_eps_{epsilon}_steps_{steps}_labeled", arr)
# np.savez(f"boosted_diff_256_eps_{epsilon}_steps_{steps}", arr)

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
arr.shape, arr.min(), arr.max()

((50000, 256, 256, 3), 0, 255)

ADM-G+U : $\epsilon = 1.5$, BIGRoC$_{PL}$

In [ ]:
!python ./gdrive/MyDrive/models_and_scripts/evaluator.py ./VIRTUAL_imagenet256_labeled.npz ./boosted_diff_256_eps_1.5_steps_7.npz

2022-02-03 12:19:28.109462: W tensorflow/core/framework/op_def_util.cc:371] Op BatchNormWithGlobalNormalization is deprecated. It will cease to work in GraphDef version 9. Use tf.nn.batch_normalization().
warming up TensorFlow...
100% 1/1 [00:08<00:00,  8.91s/it]
computing reference batch activations...
100% 157/157 [00:35<00:00,  4.42it/s]
computing/reading reference batch statistics...
computing sample batch activations...
100% 782/782 [02:51<00:00,  4.55it/s]
computing/reading sample batch statistics...
Computing evaluations...
Inception Score: 249.90652465820312
FID: 3.6888102170564707
sFID: 6.072479418420016
tcmalloc: large alloc 2000003072 bytes == 0x55629ed28000 @  0x7f9ed9651001 0x7f9ed611954f 0x7f9ed6169b58 0x7f9ed616db17 0x7f9ed620c203 0x5561ca90f424 0x5561ca90f120 0x5561ca983b80 0x5561ca97e66e 0x5561ca910aba 0x5561ca97f108 0x5561ca9109da 0x5561ca97f108 0x5561ca9109da 0x5561ca97eeae 0x5561ca97e02f 0x5561ca97dd43 0x5561caa48302 0x5561caa4867d 0x5561caa48526 0x5561caa201d3 0x55

ADM-G+U : $\epsilon = 1$, BIGRoC$_{GT}$

In [ ]:
!python ./gdrive/MyDrive/models_and_scripts/evaluator.py ./VIRTUAL_imagenet256_labeled.npz ./boosted_diff_256_eps_1_steps_7_labeled.npz

2022-02-03 12:46:26.140426: W tensorflow/core/framework/op_def_util.cc:371] Op BatchNormWithGlobalNormalization is deprecated. It will cease to work in GraphDef version 9. Use tf.nn.batch_normalization().
warming up TensorFlow...
100% 1/1 [00:03<00:00,  3.24s/it]
computing reference batch activations...
100% 157/157 [00:35<00:00,  4.44it/s]
computing/reading reference batch statistics...
computing sample batch activations...
100% 782/782 [02:48<00:00,  4.64it/s]
computing/reading sample batch statistics...
Computing evaluations...
Inception Score: 246.44772338867188
FID: 3.6592902097303295
sFID: 6.081192076149364
tcmalloc: large alloc 2000003072 bytes == 0x5568dfcea000 @  0x7fcee8d4e001 0x7fcee581654f 0x7fcee5866b58 0x7fcee586ab17 0x7fcee5909203 0x55680b2a7424 0x55680b2a7120 0x55680b31bb80 0x55680b31666e 0x55680b2a8aba 0x55680b317108 0x55680b2a89da 0x55680b317108 0x55680b2a89da 0x55680b316eae 0x55680b31602f 0x55680b315d43 0x55680b3e0302 0x55680b3e067d 0x55680b3e0526 0x55680b3b81d3 0x55

ADM-G+U : $\epsilon = 1.5$, BIGRoC$_{GT}$

In [ ]:
!python ./gdrive/MyDrive/models_and_scripts/evaluator.py ./VIRTUAL_imagenet256_labeled.npz ./boosted_diff_256_eps_1.5_steps_7_labeled.npz

downloading InceptionV3 model...
11679it [00:04, 2424.14it/s]
2022-02-02 15:37:26.783406: W tensorflow/core/framework/op_def_util.cc:371] Op BatchNormWithGlobalNormalization is deprecated. It will cease to work in GraphDef version 9. Use tf.nn.batch_normalization().
warming up TensorFlow...
100% 1/1 [00:09<00:00,  9.18s/it]
computing reference batch activations...
100% 157/157 [00:35<00:00,  4.44it/s]
computing/reading reference batch statistics...
computing sample batch activations...
100% 782/782 [02:48<00:00,  4.64it/s]
computing/reading sample batch statistics...
Computing evaluations...
Inception Score: 260.01898193359375
FID: 3.628389924147484
sFID: 6.059783066102682
tcmalloc: large alloc 2000003072 bytes == 0x55e878a94000 @  0x7f28a227b001 0x7f289ed4354f 0x7f289ed93b58 0x7f289ed97b17 0x7f289ee36203 0x55e7a5528544 0x55e7a5528240 0x55e7a559c627 0x55e7a5596ced 0x55e7a5529bda 0x55e7a5597c0d 0x55e7a5529afa 0x55e7a5597c0d 0x55e7a5529afa 0x55e7a5597915 0x55e7a55969ee 0x55e7a55966f3 0x5

ADM-G+U : $\epsilon = 3$, BIGRoC$_{GT}$

In [ ]:
!python ./gdrive/MyDrive/models_and_scripts/evaluator.py ./VIRTUAL_imagenet256_labeled.npz ./boosted_diff_256_eps_3_steps_7_labeled.npz

2022-02-01 16:26:54.342603: W tensorflow/core/framework/op_def_util.cc:371] Op BatchNormWithGlobalNormalization is deprecated. It will cease to work in GraphDef version 9. Use tf.nn.batch_normalization().
warming up TensorFlow...
100% 1/1 [00:07<00:00,  7.71s/it]
computing reference batch activations...
100% 157/157 [00:33<00:00,  4.66it/s]
computing/reading reference batch statistics...
computing sample batch activations...
100% 782/782 [02:41<00:00,  4.84it/s]
computing/reading sample batch statistics...
Computing evaluations...
Inception Score: 299.2675476074219
FID: 3.819958118079228
sFID: 5.971459498910576
tcmalloc: large alloc 2000003072 bytes == 0x562fac810000 @  0x7f856b5a9001 0x7f856807154f 0x7f85680c1b58 0x7f85680c5b17 0x7f8568164203 0x562ed7aa7544 0x562ed7aa7240 0x562ed7b1b627 0x562ed7b15ced 0x562ed7aa8bda 0x562ed7b16c0d 0x562ed7aa8afa 0x562ed7b16c0d 0x562ed7aa8afa 0x562ed7b16915 0x562ed7b159ee 0x562ed7b156f3 0x562ed7bdf4c2 0x562ed7bdf83d 0x562ed7bdf6e6 0x562ed7bb7163 0x562e

ADM-G+U : $\epsilon = 5$, BIGRoC$_{GT}$

In [ ]:
!python ./gdrive/MyDrive/models_and_scripts/evaluator.py ./VIRTUAL_imagenet256_labeled.npz ./boosted_diff_256_eps_5_steps_7_labeled.npz

2022-02-02 18:07:02.708024: W tensorflow/core/framework/op_def_util.cc:371] Op BatchNormWithGlobalNormalization is deprecated. It will cease to work in GraphDef version 9. Use tf.nn.batch_normalization().
warming up TensorFlow...
100% 1/1 [00:08<00:00,  8.79s/it]
computing reference batch activations...
100% 157/157 [00:35<00:00,  4.44it/s]
computing/reading reference batch statistics...
computing sample batch activations...
100% 782/782 [02:51<00:00,  4.55it/s]
computing/reading sample batch statistics...
Computing evaluations...
Inception Score: 346.3348693847656
FID: 4.459977279652321
sFID: 5.808254440696146
tcmalloc: large alloc 2000003072 bytes == 0x56286d484000 @  0x7fc4ff6d6001 0x7fc4fc19e54f 0x7fc4fc1eeb58 0x7fc4fc1f2b17 0x7fc4fc291203 0x562798990544 0x562798990240 0x562798a04627 0x5627989feced 0x562798991bda 0x5627989ffc0d 0x562798991afa 0x5627989ffc0d 0x562798991afa 0x5627989ff915 0x5627989fe9ee 0x5627989fe6f3 0x562798ac84c2 0x562798ac883d 0x562798ac86e6 0x562798aa0163 0x5627

$\epsilon = 3$, BIGRoC$_{GT}$

In [ ]:
!python ./gdrive/MyDrive/models_and_scripts/evaluator.py ./VIRTUAL_imagenet256_labeled.npz ./boosted_diff_256_eps_3_steps_7_labeled.npz

downloading InceptionV3 model...
11679it [00:04, 2878.08it/s]
2022-01-16 18:20:17.612924: W tensorflow/core/framework/op_def_util.cc:371] Op BatchNormWithGlobalNormalization is deprecated. It will cease to work in GraphDef version 9. Use tf.nn.batch_normalization().
warming up TensorFlow...
100% 1/1 [00:09<00:00,  9.18s/it]
computing reference batch activations...
100% 157/157 [00:35<00:00,  4.42it/s]
computing/reading reference batch statistics...
computing sample batch activations...
100% 782/782 [02:51<00:00,  4.56it/s]
computing/reading sample batch statistics...
Computing evaluations...
Inception Score: 259.0693359375
FID: 3.984601585595442
sFID: 5.0033157549130465
tcmalloc: large alloc 2000003072 bytes == 0x55a7e8426000 @  0x7f16874d6001 0x7f1683f9e54f 0x7f1683feeb58 0x7f1683ff2b17 0x7f1684091203 0x55a7157ad544 0x55a7157ad240 0x55a715821627 0x55a71581bced 0x55a7157aebda 0x55a71581cc0d 0x55a7157aeafa 0x55a71581cc0d 0x55a7157aeafa 0x55a71581c915 0x55a71581b9ee 0x55a71581b6f3 0x55a7

$\epsilon = 3$, BIGRoC$_{PL}$

In [ ]:
!python ./gdrive/MyDrive/models_and_scripts/evaluator.py ./VIRTUAL_imagenet256_labeled.npz ./boosted_diff_256_eps_3_steps_7.npz

2022-01-15 06:53:53.197560: W tensorflow/core/framework/op_def_util.cc:371] Op BatchNormWithGlobalNormalization is deprecated. It will cease to work in GraphDef version 9. Use tf.nn.batch_normalization().
warming up TensorFlow...
100% 1/1 [00:07<00:00,  7.88s/it]
computing reference batch activations...
100% 157/157 [00:34<00:00,  4.60it/s]
computing/reading reference batch statistics...
computing sample batch activations...
100% 782/782 [02:45<00:00,  4.72it/s]
computing/reading sample batch statistics...
Computing evaluations...
Inception Score: 239.62173461914062
FID: 4.072115726267782
sFID: 5.021853337463654
tcmalloc: large alloc 2000003072 bytes == 0x556d0e7a6000 @  0x7fcc10ac7001 0x7fcc0d58f54f 0x7fcc0d5dfb58 0x7fcc0d5e3b17 0x7fcc0d682203 0x556c3c298544 0x556c3c298240 0x556c3c30c627 0x556c3c306ced 0x556c3c299bda 0x556c3c307c0d 0x556c3c299afa 0x556c3c307c0d 0x556c3c299afa 0x556c3c307915 0x556c3c3069ee 0x556c3c3066f3 0x556c3c3d04c2 0x556c3c3d083d 0x556c3c3d06e6 0x556c3c3a8163 0x556

Additional Results

$\epsilon = 1.5$, BIGRoC$_{PL}$

In [ ]:
!python ./gdrive/MyDrive/models_and_scripts/evaluator.py ./VIRTUAL_imagenet256_labeled.npz ./boosted_diff_256_eps_1_half_steps_7.npz

downloading InceptionV3 model...
11679it [00:04, 2779.05it/s]
2022-01-15 06:36:24.564140: W tensorflow/core/framework/op_def_util.cc:371] Op BatchNormWithGlobalNormalization is deprecated. It will cease to work in GraphDef version 9. Use tf.nn.batch_normalization().
warming up TensorFlow...
100% 1/1 [00:08<00:00,  8.52s/it]
computing reference batch activations...
100% 157/157 [00:32<00:00,  4.80it/s]
computing/reading reference batch statistics...
computing sample batch activations...
100% 782/782 [02:44<00:00,  4.77it/s]
computing/reading sample batch statistics...
Computing evaluations...
Inception Score: 214.6321563720703
FID: 4.102903153513239
sFID: 5.112735536421496
tcmalloc: large alloc 2000003072 bytes == 0x55b63b818000 @  0x7f1762c55001 0x7f175f71d54f 0x7f175f76db58 0x7f175f771b17 0x7f175f810203 0x55b5671e3544 0x55b5671e3240 0x55b567257627 0x55b567251ced 0x55b5671e4bda 0x55b567252c0d 0x55b5671e4afa 0x55b567252c0d 0x55b5671e4afa 0x55b567252915 0x55b5672519ee 0x55b5672516f3 0x55

$\epsilon = 5$, BIGRoC$_{PL}$

In [ ]:
!python ./gdrive/MyDrive/models_and_scripts/evaluator.py ./VIRTUAL_imagenet256_labeled.npz ./boosted_diff_256_eps_5_steps_7.npz

2022-01-14 17:59:40.101140: W tensorflow/core/framework/op_def_util.cc:371] Op BatchNormWithGlobalNormalization is deprecated. It will cease to work in GraphDef version 9. Use tf.nn.batch_normalization().
warming up TensorFlow...
100% 1/1 [00:08<00:00,  8.78s/it]
computing reference batch activations...
100% 157/157 [00:35<00:00,  4.44it/s]
computing/reading reference batch statistics...
computing sample batch activations...
100% 782/782 [02:48<00:00,  4.64it/s]
computing/reading sample batch statistics...
Computing evaluations...
Inception Score: 275.64508056640625
FID: 4.450068382755262
sFID: 4.9191934585393255
tcmalloc: large alloc 2000003072 bytes == 0x55e82ff6a000 @  0x7fde93272001 0x7fde8fd3a54f 0x7fde8fd8ab58 0x7fde8fd8eb17 0x7fde8fe2d203 0x55e75ad72544 0x55e75ad72240 0x55e75ade6627 0x55e75ade0ced 0x55e75ad73bda 0x55e75ade1c0d 0x55e75ad73afa 0x55e75ade1c0d 0x55e75ad73afa 0x55e75ade1915 0x55e75ade09ee 0x55e75ade06f3 0x55e75aeaa4c2 0x55e75aeaa83d 0x55e75aeaa6e6 0x55e75ae82163 0x55